In [222]:
import requests
import numpy as np
import pandas as pd
import json

In [223]:
params = {
    'q': 'tesla',
    'tweet_mode': 'extended',
    'lang': 'en',
    'count': '100'
}

In [224]:
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAD5IUwEAAAAAYRmAUWOXGmKinw5EJY4c0mK4L%2Fk%3DfQ4d0x6AHSjU0HYYbya8lnLEE5QRsIyWakBJmiTVqscmtYEYHi'

In [225]:
response = requests.get(
    'https://api.twitter.com/1.1/search/tweets.json',
    params=params,
    headers={'authorization': 'Bearer '+BEARER_TOKEN}
)

In [226]:
def get_data(tweet):
    data = {'id':tweet['id_str'],
           'created_at':tweet['created_at'],
           'text':tweet['full_text']}
    return data

In [227]:
df = pd.DataFrame()

In [228]:
response.json()

{'statuses': [{'created_at': 'Tue Oct 19 19:45:17 +0000 2021',
   'id': 1450548626426515460,
   'id_str': '1450548626426515460',
   'full_text': 'Breaking news! It is a cool annual giveaway ! #doge #dogecoin #btc #bitcoin\n\nhttps://t.co/sf7hxbKE3Z',
   'truncated': False,
   'display_text_range': [0, 100],
   'entities': {'hashtags': [{'text': 'doge', 'indices': [46, 51]},
     {'text': 'dogecoin', 'indices': [52, 61]},
     {'text': 'btc', 'indices': [62, 66]},
     {'text': 'bitcoin', 'indices': [67, 75]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/sf7hxbKE3Z',
      'expanded_url': 'https://tesla-present.com/?1267555850',
      'display_url': 'tesla-present.com/?1267555850',
      'indices': [77, 100]}]},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to

In [229]:
for tweet in response.json()['statuses']:
    row = get_data(tweet)
    df = df.append(row,ignore_index=True)

In [230]:
df.head()

,created_at,id,text
0,Tue Oct 19 19:45:17 +0000 2021,1450548626426515460,Breaking news! It is a cool annual giveaway ! ...
1,Tue Oct 19 19:45:15 +0000 2021,1450548620479115269,RT @CryptoJeff69: ‼️THIS JUST IN‼️ Got a DM fr...
2,Tue Oct 19 19:45:14 +0000 2021,1450548616586792965,RT @johnkrausphotos: Shared with permission on...
3,Tue Oct 19 19:45:10 +0000 2021,1450548597792124928,@DesiDadChef @Anthonyvictor15 @elonmusk @adamh...
4,Tue Oct 19 19:45:08 +0000 2021,1450548592272461827,RT @bgrahamdisciple: Prediction: Tesla bulls ...


In [231]:
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-10-19 12:45:50,064 loading file /Users/roji/.flair/models/sentiment-en-mix-distillbert_4.pt


In [232]:
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet

[nltk_data] Downloading package words to /Users/roji/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [233]:
df['text'] = df['text'].map(lambda x : cleaner(x))

In [234]:
df.head(5)

,created_at,id,text
0,Tue Oct 19 19:45:17 +0000 2021,1450548626426515460,Breaking news ! It is a cool annual giveaway !...
1,Tue Oct 19 19:45:15 +0000 2021,1450548620479115269,: ‼️ THIS JUST IN ‼️ Got a from on my .
2,Tue Oct 19 19:45:14 +0000 2021,1450548616586792965,: with permission on behalf of ’ s family : “ ...
3,Tue Oct 19 19:45:10 +0000 2021,1450548597792124928,Breaking News : and go to war over 😂
4,Tue Oct 19 19:45:08 +0000 2021,1450548592272461827,: Prediction : will soon be bull - why Musk ha...


In [235]:
probs =[]
sentiments = []

In [236]:
for tweet in df['text'].to_list():
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

In [237]:
df['prob'] = probs

In [238]:
df['sentiment'] = sentiments

In [239]:
df.head(5)

,created_at,id,text,prob,sentiment
0,Tue Oct 19 19:45:17 +0000 2021,1450548626426515460,Breaking news ! It is a cool annual giveaway !...,0.998671,POSITIVE
1,Tue Oct 19 19:45:15 +0000 2021,1450548620479115269,: ‼️ THIS JUST IN ‼️ Got a from on my .,0.733252,NEGATIVE
2,Tue Oct 19 19:45:14 +0000 2021,1450548616586792965,: with permission on behalf of ’ s family : “ ...,0.980591,POSITIVE
3,Tue Oct 19 19:45:10 +0000 2021,1450548597792124928,Breaking News : and go to war over 😂,0.983553,NEGATIVE
4,Tue Oct 19 19:45:08 +0000 2021,1450548592272461827,: Prediction : will soon be bull - why Musk ha...,0.979868,NEGATIVE


In [240]:
from datetime import datetime, timedelta

In [241]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': '(tesla OR tsla OR elon musk) (lang:en)',
    'max_results': '100',
    'tweet.fields': 'created_at,lang'
}


In [242]:
dtformat = '%Y-%m-%dT%H:%M:%SZ'

In [243]:
def time_travel(now, mins):
    now = datetime.strptime(now,dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

In [244]:
now = datetime.now()  # get the current datetime, this is our starting point
last_week = now - timedelta(days=6)  # datetime one week ago = the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API

In [245]:
def get_data_seven_day(tweet):
    data = {'id':tweet['id'],
           'created_at':tweet['created_at'],
           'text':tweet['text']}
    return data

In [246]:

df = pd.DataFrame()  # initialize dataframe to store tweets
while True:
    if datetime.strptime(now, dtformat) < last_week:
        # if we have reached 7 days ago, break the loop
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    params['start_time'] = pre60
    params['end_time'] = now
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    for tweet in response.json()['data']:
        row = get_data_seven_day(tweet)  # we defined this function earlier
        df = df.append(row, ignore_index=True)

In [247]:
df['text'] = df['text'].map(lambda x : cleaner(x))

In [250]:
response.json()

{'data': [{'lang': 'en',
   'created_at': '2021-10-13T13:47:01.000Z',
   'id': '1448284141301207040',
   'text': 'RT @alex_avoigt: Per car sold, Tesla invests the most in research and development and the least in advertising\n\nCustomers appreciate that h…'},
  {'lang': 'en',
   'created_at': '2021-10-13T13:47:01.000Z',
   'id': '1448284138264424456',
   'text': 'RT @bevedoni: hey Tesla Killers,\n\nyou can not kill Tesla anymore. You should try to survive now. \n\nBest Regards\nEveryone. \n\n$TSLA'},
  {'lang': 'en',
   'created_at': '2021-10-13T13:47:01.000Z',
   'id': '1448284138230923273',
   'text': '@elonmusk Idea: Supercomputing Tesla truck factory: A widely distributed system, each truck would carry a supercomputer, altogether connected; The entire distribution would form a network of computing power; Together with AI, robotics &amp; self-driving, a worldwide Tesla moving net.'},
  {'lang': 'en',
   'created_at': '2021-10-13T13:46:59.000Z',
   'id': '1448284131645800453',
   'te

In [ ]:
import yfinance as yf
data = yf.download('TSLA',start="2021-10-12", end="2021-10-18")
tsla = yf.Ticker("TSLA")
print(data)

In [ ]:
tsla_stock = tsla.history(
    start=(data['Open'].min()).strftime('%Y-%m-%d'),
    end=data['Open'].max().strftime('%Y-%m-%d'),
    interval='60m'
).reset_index()